### Импорты


In [1]:
import pandas as pd
import copy

### Класс автобуса

In [2]:
class Bus():
    def __init__(self,busID, free = True, direction = True):
        self.busID = busID
        self.free = free
        #true - forward, false - back
        #Прямо, Обратно, Свободный
        self.direction = direction
        self.first = False

    def change_free(self):
        self.free = not(self.free)

    def change_dir(self):
        self.direction = not(self.direction)
        
    def set_time(self, time):
        self.timefree = time
        
    def first_start(self):
        self.first = True
        
    




In [3]:
class Station():
    def __init__(self, timestart, timestop, direction):
        self.timestart = timestart
        self.timestop = timestop
        self.direction = direction
    def print_info(self):
        start = self.timestart.strftime("%X")
        stop = self.timestop.strftime("%X")
        return start, stop, self.direction

### Подготовка


In [4]:
df_timetable = pd.read_excel('test.xlsx')


In [5]:
#Переведем время в datetime
df_timetable["time_dt"] = df_timetable["time"].apply(lambda x: pd.to_datetime(x, format= "%H:%M"))
print(df_timetable["time_dt"].head(5))


0   1900-01-01 03:00:00
1   1900-01-01 03:30:00
2   1900-01-01 04:00:00
3   1900-01-01 04:30:00
4   1900-01-01 05:00:00
Name: time_dt, dtype: datetime64[ns]


In [6]:
#Заменим нет рейсов на 0
df_timetable.loc[df_timetable["delayf"] == "нет рейсов", "delayf"] = 0
df_timetable.loc[df_timetable["delayb"] == "нет рейсов", "delayb"] = 0
df_timetable.head(2)

,time,countf,delayf,countb,delayb,sumf,onef,sumb,oneb,time_dt
0,03:00,0,0,0,0,0.00,0.00,0.0,0.0,1900-01-01 03:00:00
1,03:30,0,0,0,0,0.32,0.32,0.3,0.3,1900-01-01 03:30:00


### MainTest


#### Функции:


In [33]:
def addToTT(bus, countbus, dictt, ntime, minute, second, direction):
    stoptime = ntime + pd.Timedelta(minutes = minute, seconds = second)
    if stoptime.day == 2:
            stoptime -= pd.Timedelta(days = 1)
    st = Station(ntime, stoptime, direction)
    dictt[bus.busID].append(st)
    if direction != "нулевой пробег":
        bus.change_free()
    lastsendf = ntime
    countbus += 1
    return countbus, lastsendf


def changePhase(ntime, df_timetable):
    row = df_timetable.loc[df_timetable["time_dt"] == ntime]
    cbf = int(row["countf"])
    cbb = int(row["countb"])
    delf = int(row["delayf"])
    delb = int(row["delayb"])
    nextidx = row.index.to_list()[0]+1
    if nextidx >= df_timetable.shape[0]:
        nextidx = df_timetable.shape[0]-1
    return cbf, cbb, delf, delb, nextidx

def changeNextPhase(index, df_timetable):
    row = df_timetable.loc[index]
    time = row["time_dt"]
    cbf = int(row["countf"])
    cbb = int(row["countb"])
    nextdel = int(row["delayf"])
    
    return cbf, cbb, nextdel
    
    
def addLast(dictt, addtime):
    for key in dictt.keys():
        lastend = dictt[key][-1].timestop
        start = lastend + pd.Timedelta(minutes=5)
        end = start + pd.Timedelta(minutes=addtime)
        st = Station(start, end, "нулевой пробег")
        dictt[key].append(st)
        
        
        
    

In [34]:
# Расчитаем среднее время проезда в обоих направлениях
#Средняя скорость автобуса 19 км/ч
avetimef = round(6.48/19*60, 2)
avetimeb = 5.63/19*60
avtfm = avetimef // 1
# avtfs = avetimef % 1 * 60  // 1
avtbm = avetimeb // 1
# avtbs = avetimeb % 1 * 60  // 1
avtfs = 0
avtbs = 0

avtfm = 30
avtbm = 30
print(avtfm, avtfs, avtbm, avtbs)

mintofirst = 25

30 0 30 0


In [35]:
dirtt = {} #словарь со списками точек входа/выхода для каждого автобуса

buspark = [] #парк автобусов
for i in range(8):
    buspark.append(Bus(i+1))
    dirtt[i+1] = []
print(dirtt)

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}


In [36]:
ftime = pd.to_datetime(df_timetable["time"][0], format= "%H:%M")

busline = 0


busonline = []




nowbusf = 0
nowbusb = 0

lastsendf = pd.to_datetime("0:0", format= "%H:%M")
lastsendb = pd.to_datetime("0:0", format= "%H:%M")
lastfirst = pd.to_datetime("0:0", format= "%H:%M")
for i in range(24):
    for j in range(0, 60, 3):
        #нынешнее время
        ntime = ftime + pd.Timedelta(hours = i, minutes=j)
        if ntime.day == 2:
            ntime -= pd.Timedelta(days = 1)
            
        #смена фазы
        if df_timetable["time_dt"].isin([ntime]).any():
            cbf, cbb, delf, delb, nextidx = changePhase(ntime, df_timetable)
        
        #время в минутах после последней отправки
        deltasendf = (ntime - lastsendf).seconds/60
        deltasendb = (ntime - lastsendb).seconds/60
        deltafirst = (ntime - lastfirst).seconds/60
        
        ncbf, ncbb, nextdel = changeNextPhase(nextidx, df_timetable)
        #нулевой пробег
        if busline < ncbf+ncbb:
            for j in buspark:
                deltafirst = (ntime - lastfirst).seconds/60
                if (deltafirst - nextdel) >= 0 and not(j.first):
                    busline, lastfirst = addToTT(j, busline, dirtt, ntime, mintofirst, 0, "нулевой пробег")
                    j.first_start()
                if busline >= ncbf+ncbb:
                    break
                
            
            
            
            
        
        
        #вычет
        if (nowbusf + nowbusb) != 0:
            for j in buspark:
                if not(j.free):
                    if dirtt[j.busID][-1].timestop < ntime:
                        j.change_free()
                        if j.direction:
                            nowbusf -= 1

                        else: 
                            nowbusb -= 1 
                        j.change_dir()
        #добавление
        if (nowbusf + nowbusb) < (cbf + cbb) :
            #переделать в отдельную функцию
            for j in buspark:
                deltasendf = (ntime - lastsendf).seconds/60
                deltasendb = (ntime - lastsendb).seconds/60
                if j.free:
                    if nowbusf != cbf and j.direction and (deltasendf - delf) >= 0:
                        nowbusf, lastsendf = addToTT(j, nowbusf, dirtt, ntime, avtfm, avtfs, True)

                    elif nowbusb != cbb and not(j.direction) and (deltasendb - delb) >= 0:
                        nowbusb, lastsendb = addToTT(j, nowbusb, dirtt, ntime, avtbm, avtbs, False)
        
                
            
#         print(ntime, nowbusf, nowbusb, cbf, cbb, deltasendf, deltasendb, delf, delb)
                        
addLast(dirtt, mintofirst)
            

                    

                        
            
            
            

            

In [37]:
test2 = {}
for k,v in dirtt.items():
    test2[k] = []
    for i in dirtt[k]:
        test2[k].append(i.print_info())
#         print(k, *i.print_info())


In [38]:
df2 = pd.DataFrame(columns=['busID', 'TimeStart', 'TimeStop','Direction'])
for k,v in dirtt.items():
    for i in dirtt[k]:
        info = i.print_info()
        new_row = [[k, info[0],info[1],info[2]]]
        df2 = pd.concat([df2, pd.DataFrame(new_row, columns=df2.columns)], ignore_index=True)
        
df2.head(5)


,busID,TimeStart,TimeStop,Direction
0,1,04:30:00,04:55:00,нулевой пробег
1,1,05:00:00,05:30:00,True
2,1,05:33:00,06:03:00,False
3,1,06:09:00,06:39:00,True
4,1,06:42:00,07:12:00,False


In [39]:
df2.to_excel('test11.xlsx')